In [1]:
# Adding Self Reflection and Memory to Our AI Agent..

In [2]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=api_key
)

In [5]:
system_prompt = "Answer all user questions"
user_prompt = "What have I asked?"
response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.0,
    messages=[
        {"role" : "system", "content":system_prompt},
        {"role" : "user", "content" : user_prompt},
    ]
)

In [6]:
response.choices[0].message.content

"I don't have access to previous interactions or any specific questions you've asked before. However, I'm here to help with any questions you have now! What would you like to know?"

In [8]:
#Adding Memory

memory = [
     {"role" : "system", "content": "Answer all the questions"},
        {"role" : "user", "content" : "What's an LLM"},
]

In [9]:
new_response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)
memory.append(
    {"role":"assistant", "content": new_response.choices[0].message.content}
)
memory

[{'role': 'system', 'content': 'Answer all the questions'},
 {'role': 'user', 'content': "What's an LLM"},
 {'role': 'assistant',
  'content': "An LLM, or Large Language Model, is a type of artificial intelligence model designed to understand and generate human language. These models are trained on vast amounts of text data and use deep learning techniques, particularly neural networks, to learn patterns, grammar, context, and even some level of reasoning in language. LLMs can perform a variety of tasks, including text generation, translation, summarization, and answering questions, among others. Examples of LLMs include OpenAI's GPT-3 and GPT-4, Google's BERT, and others."}]

In [10]:
memory.append(
    {"role": "user", "content": "What have I asked?"}
)
memory

[{'role': 'system', 'content': 'Answer all the questions'},
 {'role': 'user', 'content': "What's an LLM"},
 {'role': 'assistant',
  'content': "An LLM, or Large Language Model, is a type of artificial intelligence model designed to understand and generate human language. These models are trained on vast amounts of text data and use deep learning techniques, particularly neural networks, to learn patterns, grammar, context, and even some level of reasoning in language. LLMs can perform a variety of tasks, including text generation, translation, summarization, and answering questions, among others. Examples of LLMs include OpenAI's GPT-3 and GPT-4, Google's BERT, and others."},
 {'role': 'user', 'content': 'What have I asked?'}]

In [11]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all the questions'},
 {'role': 'user', 'content': "What's an LLM"},
 {'role': 'assistant',
  'content': "An LLM, or Large Language Model, is a type of artificial intelligence model designed to understand and generate human language. These models are trained on vast amounts of text data and use deep learning techniques, particularly neural networks, to learn patterns, grammar, context, and even some level of reasoning in language. LLMs can perform a variety of tasks, including text generation, translation, summarization, and answering questions, among others. Examples of LLMs include OpenAI's GPT-3 and GPT-4, Google's BERT, and others."},
 {'role': 'user', 'content': 'What have I asked?'},
 {'role': 'assistant',
  'content': 'You asked, "What\'s an LLM?" which refers to a Large Language Model in the context of artificial intelligence and natural language processing.'}]

In [14]:
#Create a memory layer

class Memory:
    def __init__(self):
        self._messages = []

    def add_message(self, role, content):
        self._messages.append({"role": role, "content": content})

    def get_messages(self):
        return self._messages

    def last_message(self):
        if self._messages:
            return self._messages[-1]



In [15]:
SELF_CRITIQUE_PROMPT = """
Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. 
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
"""

In [ ]:
class Agent:
    """A self-reflection AI Agent"""

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature

        self.client = client

        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions}",
        )

        self.critique_prompt = SELF_CRITIQUE_PROMPT
    



    
    def invoke(self, 
               user_message: str, 
               self_reflection: bool = False, 
               max_iter: int = 1, 
               verbose: bool = False) -> str:
    
        # Rules
        # - Don't allow values less than 1
        # - Don't allow values greater than 3
        # - Max iter is controlled by self_reflection flag. 
        # - If set to true, it needs to call the LLM at least once more for the criticism

        self.memory.add_message(
            role="user",
            content=user_message
        )
        if verbose:
            self._log_last_message()

        max_iter = max_iter if max_iter >= 1 else 1
        max_iter = max_iter if max_iter <= 3 else 3
        max_iter = max_iter if self_reflection else 0.5
        loops = 2 * max_iter

        for i in range(loops):
            ai_message = self._get_completion(
                messages = self.memory.get_messages()
            )

            self.memory.add_message(
                role = "assistant",
                content = ai_message.content,
            )
            if verbose:
                self._log_last_message()

            if i < loops - 1:
                self.memory.add_message(
                    role = "user", 
                    content = self.critique_prompt
                )
                if verbose:
                    self._log_last_message()

                ai_message = self._get_completion(
                    messages = self.memory.get_messages()
                )

    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:

        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages
        )
        
        return response.choices[0].message

    def _log_last_message(self):
        print(f"### {self.memory.last_message()['role']} message ###\n".upper())
        print(f"{self.memory.last_message()['content']} \n")
        print("\n________________________________________________________________\n")


In [19]:
agent = Agent()
agent.invoke(
    user_message="Pick only one. Who is the best character in Game of Thrones?",
    self_reflection=True,
    verbose=True,
)

### USER MESSAGE ###

Pick only one. Who is the best character in Game of Thrones? 


________________________________________________________________

### ASSISTANT MESSAGE ###

Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and moral complexity. He navigates the treacherous political landscape with a unique perspective and often serves as a voice of reason amidst chaos. Who is your favorite character? 


________________________________________________________________

### USER MESSAGE ###


Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. 
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
 


_________________________________________________

In [20]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI Agent, your role is Personal Assistant, and you need to Help users with any question"},
 {'role': 'user',
  'content': 'Pick only one. Who is the best character in Game of Thrones?'},
 {'role': 'assistant',
  'content': 'Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and moral complexity. He navigates the treacherous political landscape with a unique perspective and often serves as a voice of reason amidst chaos. Who is your favorite character?'},
 {'role': 'user',
  'content': '\nReflect on your previous response...\nIdentify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. \nProvide a revised response if necessary in a Json Output structure:\n{\n    "original_response": "",\n    "revisions_needed": "",\n    "updated_response": ""\n}\n'},
 {'role

In [21]:
json.loads(agent.memory.last_message()["content"])["updated_response"]

"Many fans consider Tyrion Lannister the best character in 'Game of Thrones' due to his wit, intelligence, and moral complexity, making him a compelling figure in the series."